In [1]:
import sys  
import os
import pandas as pd
import io
sys.path.insert(0, '../../../../../scripts/')
sys.path.insert(0, '../../../../../dataset/')
import ngrams_scikitlearn
import hate_column_converter
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.naive_bayes import GaussianNB
from imblearn.under_sampling import RandomUnderSampler

In [2]:
dataframe = pd.read_csv('../../../../../dataset/HateBR.csv')
new_dataframe = hate_column_converter.convert_to_binary(dataframe)
instagram_comments = new_dataframe['instagram_comments']
y = new_dataframe['hate_speech']
ndf = new_dataframe[new_dataframe['hate_speech'] > 0]

/workspaces/hate-speech-detection-python/train/notebooks-base-paper/nb/hate-speech/gaussian/../../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  lines_with_two_numbers_in_hate_speech[['col1', 'col2']] = lines_with_two_numbers_in_hate_speech['hate_speech'].str.split(',', expand=True)
/workspaces/hate-speech-detection-python/train/notebooks-base-paper/nb/hate-speech/gaussian/../../../../../scripts/hate_column_converter.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-co

In [3]:
X = ngrams_scikitlearn.get_vector(instagram_comments)

undersample = RandomUnderSampler(sampling_strategy='majority')
x_over, y_over = undersample.fit_resample(X, y)

In [4]:
x_train, x_test, y_train, y_test = train_test_split(x_over, y_over, train_size=0.8, test_size=0.1)

In [5]:
mnb = GaussianNB()
model = mnb.fit(x_train.toarray(), y_train)
y_pred = model.predict(x_test.toarray())

print('F1 score 1:', f1_score(y_test, y_pred, labels=[0,1], pos_label=1))
print('F1 score 0:', f1_score(y_test, y_pred, labels=[0,1], pos_label=0))
print('F1 score todos:', f1_score(y_test, y_pred, labels=[0,1], average='macro'))

F1 score 1: 0.6993006993006994
F1 score 0: 0.7114093959731544
F1 score todos: 0.7053550476369269


In [7]:
import joblib

model_filename = '../../../../../models/generated-models-base-paper/offensive_nb_gaussian_tfidf.pkl'
joblib.dump(model, model_filename)

['../../../../../models/generated-models-base-paper/offensive_nb_gaussian_tfidf.pkl']